# Practice Session 08: Data streams


Author: <font color="blue">Manvir Kaur Singh</font>

E-mail: <font color="blue">manvir.kaur01@estudiant.upf.edu</font>

Date: <font color="blue">3/12/2023</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip


# 0. Dataset and how to iterate

In [2]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(text):
                          
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [4]:
# Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'her'
Current word 'dangerous'
Current word 'your'
Current word 'the'
Current word 'absolute'
Current word 'kids'
Current word 'that'
Current word 'room'
Current word 'for'
Current word 'well'
Current word 'thank'
Current word 'it'
Current word 'see'
Current word 'closes'
Current word 'been'
- Read 100000/300000 words so far
Current word 'you'
Current word 'why'
Current word 'where'
Current word 'to'
Current word 'in'
Current word 'french'
Current word 'it'
Current word 'ring'
- Read 200000/300000 words so far
Current word 'charge'
Current word 'together'
Current word 'get'
Current word 'me'
Current word 'go'
Current word 'phones'
Current word 'would'
Current word 'one'
Current word 'stand'
Current word 'right'
Current word 'of'
Current word 'the'
Current word 'be'
Current word 'everything'
- Read 300000/300000 words so far


In [5]:
# Run this if above gives an error about 'punkt'
#nltk.download('punkt')

# 1. Determine approximately the top-10 words

<font size="+1" color="red">Implement "add_reservoir"</font>

In [6]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        random_index = random.randint(0, max_reservoir_size - 1)
        reservoir[random_index] = item
    assert(len(reservoir) <= max_reservoir_size)

<font size="+1" color="red">Implement "reservoir_sampling"</font>

In [7]:
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    
    words_read = 0
    
    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        words_read += 1
        ptocheck = random.random()
        if ptocheck > reservoir_size/words_read:
            add_to_reservoir(reservoir, word, reservoir_size)

    return (words_read, reservoir)

In [8]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000028
Number of items sampled : 1500


In [9]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

66 you
49 it
41 the
40 to
22 me
22 and
20 is
20 do
19 that
17 what


<font size="+1" color="red">Print the top items and their relative frequencies</font>

In [10]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

total_items = len(reservoir)
most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:15]
for absolute_frequency, word in most_frequent_items:
    relative_frequency = (absolute_frequency / total_items) * 100
    print("%d %s (%.2f%%)" % (absolute_frequency, word, relative_frequency))

66 you (4.40%)
49 it (3.27%)
41 the (2.73%)
40 to (2.67%)
22 me (1.47%)
22 and (1.47%)
20 is (1.33%)
20 do (1.33%)
19 that (1.27%)
17 what (1.13%)
15 your (1.00%)
15 of (1.00%)
15 in (1.00%)
14 lenny (0.93%)
14 her (0.93%)


<font size="+1" color="red">Increase the max limit of words so that one pass takes no more than 5 minutes to be completed.</font>

In [11]:
reservoir_sizes = [50, 100, 500, 1000, 2000, 4000, 6000, 8000]

for reservoir_size in reservoir_sizes:
    (items_seen, le_reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=2000000, report_every=1000000)

    freq = {}
    for item in le_reservoir:
        freq[item] = le_reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    for reservoir_freq, word in most_frequent_items:
        print('Word:', word)
        print('appeareances in reservoir:', reservoir_freq)
        absolute_frequency = reservoir_freq*(items_seen/reservoir_size)
        print('Absolute frequency (approx for entire dataset) %f' % absolute_frequency)
        print('Relative frequency (approx) %f%%' % ((absolute_frequency/(items_seen))*100))

- Read 1000000/2000000 words so far
- Read 2000000/2000000 words so far
Word: you
appeareances in reservoir: 3
Absolute frequency (approx for entire dataset) 120001.320000
Relative frequency (approx) 6.000000%
Word: to
appeareances in reservoir: 3
Absolute frequency (approx for entire dataset) 120001.320000
Relative frequency (approx) 6.000000%
Word: that
appeareances in reservoir: 3
Absolute frequency (approx for entire dataset) 120001.320000
Relative frequency (approx) 6.000000%
Word: she
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 80000.880000
Relative frequency (approx) 4.000000%
Word: know
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 80000.880000
Relative frequency (approx) 4.000000%
- Read 1000000/2000000 words so far
- Read 2000000/2000000 words so far
Word: to
appeareances in reservoir: 6
Absolute frequency (approx for entire dataset) 120001.320000
Relative frequency (approx) 6.000000%
Word: the
appeareances in rese

<font size="+1" color="red">Find by trial and error, and include in your report, the minimum reservoir size you need to have somewhat stable results</font>

After running tests, it seems that for sizes bigger than 500 the top words remain stable. Then we can suggests that a reservoir size of 500 is sufficient for stability. Stability in this context means that the top words selected by the reservoir sampling algorithm are consistent across different runs.

In [ ]:
# Perform reservoir sampling without a max_words limit
reservoir_sizes = [50, 100, 500, 1000, 2000, 4000, 6000, 8000]

for reservoir_size in reservoir_sizes:
    (items_seen, le_reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, report_every=1000000)

    freq = {}
    for item in le_reservoir:
        freq[item] = le_reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    for reservoir_freq, word in most_frequent_items:
        print('Word:', word)
        print('appeareances in reservoir:', reservoir_freq)
        absolute_frequency = reservoir_freq*(items_seen/reservoir_size)
        print('Absolute frequency (approx for entire dataset) %f' % absolute_frequency)
        print('Relative frequency (approx) %f%%' % ((absolute_frequency/(items_seen))*100))


- Read 1000000 words so far
- Read 2000000 words so far
Word: to
appeareances in reservoir: 3
Absolute frequency (approx for entire dataset) 176693.040000
Relative frequency (approx) 6.000000%
Word: your
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 117795.360000
Relative frequency (approx) 4.000000%
Word: you
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 117795.360000
Relative frequency (approx) 4.000000%
Word: with
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 117795.360000
Relative frequency (approx) 4.000000%
Word: vereker
appeareances in reservoir: 2
Absolute frequency (approx for entire dataset) 117795.360000
Relative frequency (approx) 4.000000%
- Read 1000000 words so far
- Read 2000000 words so far


<font size="+1" color="red">Brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

For a relatively stable set of top words, a reservoir size of 500 seems to be adequate as before.
As reservoir size is increased, the top words remained consistent across runs, indicating that a larger reservoir captures more representative samples from the dataset. The top words, along with their absolute and relative frequencies, show consistency in multiple runs, especially for reservoir sizes larger than 500.

# 2. Determine approximately the distinct number of words

In [ ]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [ ]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Perform the requested number of passes.</font>

In [ ]:
def distinct_word_estimates(number_of_passes):
    estimates = []

    for i in range(number_of_passes):
        
        hash_function = random_hash_function()
        
        maxR = 0

        for word in read_by_words(INPUT_FILE, report_every=2000000):
            r = count_trailing_zeroes(hash_function(word))
            maxR = max(r, maxR)
            
        estimate = 2**maxR

        estimates.append(estimate)
        print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
    return estimates

In [ ]:
estimates = distinct_word_estimates(5)

In [ ]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

<font size="+1" color="red">Remove the limit of max words, or set to a high number. </font>

In [ ]:
print('---> Runs for 10 passes')
for i in range(3):
    print('run #%d' % (i + 1))
    estimates10 = distinct_word_estimates(10)
    print("* Average of estimates: %.1f" % statistics.mean(estimates10))
    print("* Median  of estimates: %.1f" % statistics.median(estimates10))

print('---> Runs for 20 passes')
for i in range(3):
    print('run #%d' % (i + 1))
    estimates20 = distinct_word_estimates(20)
    print("* Average of estimates: %.1f" % statistics.mean(estimates20))
    print("* Median  of estimates: %.1f" % statistics.median(estimates20))

The big averages happen because really big or small numbers affect them a lot. A median, which is just the middle number, is better here because it doesn't get messed up by those really big or small numbers.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>